In [ ]:
import pandas as pd
import gzip
import ast

In [ ]:
# En este caso el dataset no cumple con un formato valido JSON donde el par clave-valor deben estar en comillas dobles.
# Se optó por usar ast.literal_eval() que puede manejar las comillas simples en lugar del modulo json.

# Ruta al dataset
path = '/content/drive/MyDrive/users_items.json.gz'

# Leemos el archivo usando ast.literal_eval para analizar la cadena JSON
data = []
with gzip.open(path, 'r') as f:
    for line in f:

        data.append(ast.literal_eval(line.decode("utf-8")))

# Convertimos a DataFrame
df_items = pd.DataFrame(data)
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [ ]:
# Damos un vistazo rapido a los tipos de datos y verificamos nulos
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [ ]:
# Observamos la columna 'items' para comprobar que su estructura de datos es un lista de diccionarios.
df_items['items'][0][:3]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 6,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 7,
  'playtime_2weeks': 0}]

In [ ]:
# Evidenciamos que efectivamente es una lista de diccionarios y procedemos a desanidarlo.
meta = df_items.columns[:4].tolist()
df_items_expanded = pd.json_normalize(data, 'items', meta)
df_items_expanded.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [ ]:
# Comprobamos datos del df desanidado.
df_items_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   user_id           object
 5   items_count       object
 6   steam_id          object
 7   user_url          object
dtypes: int64(2), object(6)
memory usage: 314.5+ MB


In [ ]:
# Notamos que ahora tiene 7 columnas y mas de 5 millones de filas.
# Vamos a comprobar duplicados y nulos para reducir un poco su tamaño.
df_items_expanded.isnull().sum()

item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
user_id             0
items_count         0
steam_id            0
user_url            0
dtype: int64

In [ ]:
# No tenemos nulos por lo que ahora comprobamos duplicados.
df_items_expanded.duplicated().sum()

59104

In [ ]:
# Tenemos duplicados que procedemos a eliminar.
df_items_expanded.drop_duplicates(inplace=True)
df_items_expanded.shape

(5094105, 8)

In [ ]:
### La columna 'playtime_2weeks' es irrelevante para nuestro proyecto ya que el tiempo acumulado de juego
# por usuario que es lo que nos interesa lo tenemos en 'playtime_forever'.

### Igualmente ocurre con 'user_url' y 'steam_id' que es un identificador unico para cada usuario en la
# plataforma Steam pero no esta presente en los demas de dataset por lo que se decide emplear 'user_id'.
columns_to_drop = ['playtime_2weeks','user_url','steam_id']
df_items_expanded.drop(columns = columns_to_drop, axis=1, inplace=True)

In [ ]:
# Cambiamos el tipo de dato de las columnas 'item_id'e 'items_count' a 'int'.
columns = ['item_id', 'items_count']
df_items_expanded[columns] = df_items_expanded[columns].astype(int)

In [ ]:
# Cambiamos el tipo de dato de las columnas 'item_id'e 'items_count' a 'int'.
columns = ['item_name', 'user_id']
df_items_expanded[columns] = df_items_expanded[columns].astype(str)

In [ ]:
# Verificamos los tipos de datos
df_items_expanded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5094105 entries, 0 to 5153208
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           int64 
 1   item_name         string
 2   playtime_forever  int64 
 3   user_id           string
 4   items_count       int64 
dtypes: int64(3), string(2)
memory usage: 233.2 MB


In [ ]:
# Exportamos a CSV.
path = 'data/user_items.csv'
df_items_expanded.to_csv(path, index=False, encoding='utf-8')
print(f'El archivo se guardó correctamente en {path}')

El archivo se guardó correctamente en data/user_items.csv
